In [1]:
import os
import numpy as np
import logging
import sys

ImportError: No module named 'numpy'

In [2]:
import mxnet as mx
from mxnet import nd
from mxnet.gluon import nn

ImportError: No module named 'mxnet'

In [3]:
import tvm
import tvm.relay as relay
from tvm.contrib import graph_runtime
# from tvm.relay.transform import FastMath

ImportError: No module named 'tvm'

In [4]:
np_weight = np.array( [[[[ 0.1, 0.2, 0.6], 
                       [ 0.1, 0.2, 0.6], 
                       [0.1, 0.2, 0.6]]]]).astype('float32')

NameError: name 'np' is not defined

In [5]:
nd_weight=nd.array(np_weight)
print(nd_weight)

NameError: name 'nd' is not defined

In [6]:
W = mx.initializer.Constant(nd_weight)
print(W)

NameError: name 'mx' is not defined

In [7]:
np_bias = np.array([0.01]).astype('float32')

NameError: name 'np' is not defined

In [1]:
nd_bias=nd.array(np_bias)
print(nd_bias)

NameError: name 'nd' is not defined

In [2]:
B = mx.initializer.Constant(nd_bias)
print(B)

NameError: name 'mx' is not defined

In [3]:
conv2d_layer = nn.Conv2D(channels=1, kernel_size=(3, 3), 
                         strides=(1, 1), padding=(1, 1), dilation=(1, 1), groups=1,
                         layout='NCHW', activation= 'relu', use_bias=True, weight_initializer=W,
                         bias_initializer=B, in_channels=0)

NameError: name 'nn' is not defined

In [4]:
print(conv2d_layer.bias.data)

NameError: name 'conv2d_layer' is not defined

In [5]:
# mxnet exp custom layer
"""
class op(nn.HybridBlock):
    def __init__(self, **kwargs):
        super(op, self).__init__(**kwargs)

    def forward(self, x):
        return x.softmax()
"""

'\nclass op(nn.HybridBlock):\n    def __init__(self, **kwargs):\n        super(op, self).__init__(**kwargs)\n\n    def forward(self, x):\n        return x.softmax()\n'

In [6]:
np_input = np.array([[[
                    [ 0.1, 0.2, 0.5, 0.1], 
                    [ 0.1, 0.2, 0.5, 0.1], 
                    [ 0.1, 0.2, 0.5, 0.1], 
                    [ 0.1, 0.2, 0.5, 0.1] 
                    ]]]).astype('float32')

NameError: name 'np' is not defined

In [7]:
mxnet_input = nd.array(np_input)
print(mxnet_input)

NameError: name 'nd' is not defined

In [ ]:
net = nn.HybridSequential()
net.add(conv2d_layer)
net.initialize()

In [ ]:
net.summary(mxnet_input)

In [ ]:
print(net)

In [ ]:
mxnet_output = net(mxnet_input)
print(mxnet_output)

In [ ]:
#tvm setting
shape_dict = {'data': np_input.shape}
#target = 'llvm -target=riscv64-unkown-elf -system-lib'
target='llvm -system-lib'
ctx = tvm.cpu(0)
dtype = 'float32'

In [ ]:
#tvm parse mxnet model
mod, params = relay.frontend.from_mxnet(net, shape_dict,)

In [ ]:
print(mod)

In [ ]:
print(params)

In [ ]:
#logging.getLogger("compile_engine").setLevel(logging.INFO)
#logging.getLogger("compile_engine").addHandler(logging.StreamHandler(sys.stdout))
with relay.build_config(opt_level=0):
    graph, lib, params = relay.build(mod, target, params=params)

In [ ]:
m = graph_runtime.create(graph, lib, ctx)
# set inputs
m.set_input('data', tvm.nd.array(np_input.astype(dtype)))
m.set_input(**params)
# execute
m.run()
# get outputs
tvm_output = m.get_output(0)

In [ ]:
print(tvm_output)

In [ ]:
#save graph. lib. params. in directory
path_ll = "./model.ll"
with open(path_ll, "w") as f:
    f.write(lib.get_source())

path_graph = "./model.json"
#path_graph = path_dir+"/model.graph"
with open(path_graph, "w") as fo:
    fo.write(graph)

path_params = "./model.params"
with open(path_params, "wb") as fo:
    fo.write(relay.save_param_dict(params))

In [ ]:
#print(graph)

In [ ]:
#print(lib.get_source())